In [1]:
# pip install openpyxl

In [2]:
import pandas as pd
import numpy as np

In [3]:
welcome = pd.read_csv("welcome_survey.csv")[["What's your PID?", "Email Address", "Assigned"]].rename(columns={"What's your PID?":'PID'})
roster = pd.read_csv("414_roster.csv")

In [4]:
welcome

,PID,Email Address,Assigned
0,A18078963,emings@ucsd.edu,3:30-3:50PM
1,A18414042,nble@ucsd.edu,4:00-4:20PM
2,A18273293,Xij018@ucsd.edu,3:00-3:20PM
3,A17980184,syramirez@ucsd.edu,3:00-3:20PM
4,A18167322,ederickson@ucsd.edu,3:30-3:50PM
...,...,...,...
135,A17123158,smoralesfermin@ucsd.edu,4:30-4:50PM
136,A18648933,aialexander@ucsd.edu,4:30-4:50PM
137,A18649002,jez020@ucsd.edu,4:30-4:50PM
138,A17772318,coj006@ucsd.edu,3:00-3:20PM


In [5]:
roster

,Name,SID,Email,Role
0,ALIYA ABULAITI,U10484150,1300141358@qq.com,Student
1,Aaron Fang,A18299213,zhf019@ucsd.edu,Student
2,Aaryaman Singhee,A17467713,asinghee@ucsd.edu,Student
3,Abraham Cruz-Arroyo,A18263721,acruzarroyo@ucsd.edu,Student
4,Abraham Grajiola,A17514281,agrajiola@ucsd.edu,Student
...,...,...,...,...
160,Yuwen Cen,A17052737,ycen@ucsd.edu,Student
161,Yuzhe Lin,A18006637,yul288@ucsd.edu,Student
162,Zaira Leal,A16599419,zpleal@ucsd.edu,Student
163,Zhaun Miguel Guerrero,A17796799,zhguerrero@ucsd.edu,Student


In [6]:
welcome.isna().sum()

PID              0
Email Address    0
Assigned         0
dtype: int64

In [7]:
# contains some missing values, so when you merge make sure you merge on something both dataframes aren't missing.
roster.isna().sum()

Name     0
SID      2
Email    0
Role     0
dtype: int64

In [8]:
df = welcome.merge(roster, how="right", left_on="Email Address", right_on="Email")[["Email", "SID", "Assigned", "Name"]]
df

,Email,SID,Assigned,Name
0,1300141358@qq.com,U10484150,4:30-4:50PM,ALIYA ABULAITI
1,zhf019@ucsd.edu,A18299213,4:30-4:50PM,Aaron Fang
2,asinghee@ucsd.edu,A17467713,3:30-3:50PM,Aaryaman Singhee
3,acruzarroyo@ucsd.edu,A18263721,4:00-4:20PM,Abraham Cruz-Arroyo
4,agrajiola@ucsd.edu,A17514281,3:30-3:50PM,Abraham Grajiola
...,...,...,...,...
160,ycen@ucsd.edu,A17052737,4:30-4:50PM,Yuwen Cen
161,yul288@ucsd.edu,A18006637,3:00-3:20PM,Yuzhe Lin
162,zpleal@ucsd.edu,A16599419,3:30-3:50PM,Zaira Leal
163,zhguerrero@ucsd.edu,A17796799,3:00-3:20PM,Zhaun Miguel Guerrero


In [9]:
# Checking to see if anyone filled the form out twice. Looks good!
df.groupby('SID').count().sort_values(by='Assigned', ascending=False).head()

,Email,Assigned,Name
SID,,,
U10484150,1,1,1
A18245775,1,1,1
A18187588,1,1,1
A18212726,1,1,1
A18221114,1,1,1


In [10]:
# # if anyone filled out the form twice, here is how to remove the duplicates!

# # Removing duplicates in case someone filled out the form twice, assuming df is named quiz_slot.
# quiz_slot = quiz_slot.groupby(['PID', 'Student', 'Email']).first()
# quiz_slot = quiz_slot.reset_index()

# ALSO: some people might not have their PID on gradescope, make sure to use welcome survey's PIDs

In [11]:
df["Assigned"] = df["Assigned"].fillna("Null")

In [12]:
# randomly assign quiz slot to those who didn't fill out the Welcome Survey. Make sure to send out an email to 
# those who haven't filled out the Welcome Survey to minimize the amount of randomization.

def find_timeslot(time):
    times = ["3:30-3:50PM", "3:00-3:20PM", "4:00-4:20PM", "4:30-4:50PM"]
    if time == "Null":
        return np.random.choice(times)
    else:
        return time

In [13]:
df['Assigned'] = df['Assigned'].apply(find_timeslot)

In [14]:
df["Assigned"].value_counts()

4:30-4:50PM    43
3:00-3:20PM    43
3:30-3:50PM    41
4:00-4:20PM    38
Name: Assigned, dtype: int64

In [15]:
def process_chart(chart_path, room_name):
    chart_df = pd.read_excel(chart_path)
    chart_df = chart_df.iloc[13:, [7, 2]]
    chart_df.columns = ['seat', 'left']
    chart_df['left'] = chart_df['left'] == 'YES'
    chart_df['room'] = room_name
    return chart_df[['room', 'seat', 'left']]

## seating charts link
https://mathweb.ucsd.edu/~gptesler/assigningseats.html

In [16]:
center_214 = process_chart('center_214.xlsx', 'Center 214')
center_214

/home/m5hong/.local/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,room,seat,left
13,Center 214,Seat: A-1,False
14,Center 214,Seat: A-2,False
15,Center 214,Seat: A-3,False
16,Center 214,Seat: A-4,True
17,Center 214,Seat: B-1,False
...,...,...,...
154,Center 214,Seat: J-14,False
155,Center 214,Seat: J-15,False
156,Center 214,Seat: J-16,False
157,Center 214,Seat: K-5,False


In [17]:
# you can do 2 things! 
# 1. independently randomize for each section: separate each section into its own DataFrame, then concatenate all of them
# after running the script
# 2. only randomize the largest section, and copy the randomized seating assignments to the other sections. 

# MAKE SURE TO CHANGE THE SEED EVERY TIME

quiz_slot_3pm = df[df['Assigned'] == '3:00-3:20PM']
quiz_slot_330pm = df[df['Assigned'] == '3:30-3:50PM']
quiz_slot_4pm = df[df['Assigned'] == '4:00-4:20PM']
quiz_slot_430pm = df[df['Assigned'] == '4:30-4:50PM']

def create_assignments(section_df, chart_df):
    np.random.seed(1)
    section_df = section_df.copy()
    chart_df = chart_df.copy()

    # To make sure people aren't assigned to left-handed desks
    # Not always possible
    if (~chart_df['left']).sum() >= section_df.shape[0]:
        chart_df = chart_df[~chart_df['left']]
        
    chart_df = chart_df.sample(frac=1).reset_index(drop=True)
    section_df = section_df.sample(frac=1).reset_index(drop=True)
    
    together = pd.concat([chart_df, section_df], axis=1).iloc[:section_df.shape[0]]
    return together

In [18]:
# here is the code to randomizing each section independently.
quiz1_seat_assignments = pd.concat([create_assignments(quiz_slot_3pm, center_214), create_assignments(quiz_slot_330pm, center_214),\
                                    create_assignments(quiz_slot_4pm, center_214), create_assignments(quiz_slot_430pm, center_214)])
quiz1_seat_assignments

,room,seat,left,Email,SID,Assigned,Name
0,Center 214,Seat: G-9,False,zhguerrero@ucsd.edu,A17796799,3:00-3:20PM,Zhaun Miguel Guerrero
1,Center 214,Seat: B-11,False,hhn007@ucsd.edu,A17453459,3:00-3:20PM,Haley Nguyen
2,Center 214,Seat: D-11,False,authompson@ucsd.edu,A17867880,3:00-3:20PM,Audrey Thompson
3,Center 214,Seat: B-3,False,swkulkarni@ucsd.edu,A18234721,3:00-3:20PM,Swara Kulkarni
4,Center 214,Seat: C-5,False,lnoguchi@ucsd.edu,A17914182,3:00-3:20PM,Leela Noguchi
...,...,...,...,...,...,...,...
38,Center 214,Seat: H-14,False,ruanshifeiucsd@163.com,U10466180,4:30-4:50PM,SHIFEI RUAN
39,Center 214,Seat: C-11,False,s2salas@ucsd.edu,A17430908,4:30-4:50PM,Sophia Salas
40,Center 214,Seat: A-5,False,l4lei@ucsd.edu,A17916922,4:30-4:50PM,Lifang Lei
41,Center 214,Seat: H-7,False,azaitsu@ucsd.edu,A18577559,4:30-4:50PM,Ace Zaitsu


In [19]:
# However, since you will need to manually separate seats from each other using this as a base, I recommend just 
# paying attention to the seats from the biggest quiz section and copying/pasting the manually spread out version to the
# other sections.

In [20]:
quiz1_seat_assignments.to_csv("quiz1.csv", index=False)

In [21]:
quiz1_seat_assignments["Assigned"].value_counts()

3:00-3:20PM    43
4:30-4:50PM    43
3:30-3:50PM    41
4:00-4:20PM    38
Name: Assigned, dtype: int64

In [22]:
# after you download the seating assignment file, you should manually spread out students. I mark down where everyone is 
# sitting and then move people around.